# Experimentation Only

In [13]:
import sys
!{sys.executable} -m pip install tqdm

You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#For Prediction
from sklearn import metrics
from sklearn import preprocessing, cross_validation
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Dense, Dropout, Input, LSTM, Activation, Flatten
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences

In [15]:
# Read Datasets 
#research_data_raw = pd.read_csv('Datasets/bitcoin_research_data_v2.csv')
research_data = pd.read_csv('Datasets/BlockChain_Train_csv.csv')
# Drop unnecessary columns
#research_data = research_data_raw.drop('Unnamed: 0', axis=1)
research_data.head()

,Date,Close,USD_Exchange_Trade_Volume,Bitcoins_in_circulation,MarketCap,BlockSize,AvgBlockSize,Orphaned_Block,TransactionsPerBlock,MedianTransactCnfmTime,...,USD/JPY,USD/CHF,USD/KRW,USD/GBP,EUR/KRW,USD/CNY,Google Trends Interest,GTrends Normalized,US Federal funds rate,Breakeven Inflation Rate
0,3-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,90.73,1.06,1269.75,0.69,1774.86,6.83,0,0.0,0.08,0.15
1,5-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,92.08,1.08,1326.15,0.69,1843.95,6.82,0,0.0,0.11,0.15
2,7-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,93.79,1.12,1305.55,0.67,1763.60,6.84,0,0.0,0.11,0.47
3,9-Jan-09,0.0,0.0,750.0,0.0,0.0,0.0,0,1.0,0.0,...,91.24,1.10,1324.45,0.66,1808.74,6.83,0,0.0,0.09,0.56
4,11-Jan-09,0.0,0.0,7600.0,0.0,0.0,0.0,0,1.0,0.0,...,91.24,1.10,1324.45,0.66,1808.74,6.83,0,0.0,0.09,0.56


In [16]:
# Convert data to milliseconds
research_data['Epoch'] = pd.to_datetime(research_data['Date'] + ' 23:59:59.000')
research_data['Epoch'] = (research_data['Epoch'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
research_data.head()

,Date,Close,USD_Exchange_Trade_Volume,Bitcoins_in_circulation,MarketCap,BlockSize,AvgBlockSize,Orphaned_Block,TransactionsPerBlock,MedianTransactCnfmTime,...,USD/CHF,USD/KRW,USD/GBP,EUR/KRW,USD/CNY,Google Trends Interest,GTrends Normalized,US Federal funds rate,Breakeven Inflation Rate,Epoch
0,3-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,1.06,1269.75,0.69,1774.86,6.83,0,0.0,0.08,0.15,1231027199
1,5-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,1.08,1326.15,0.69,1843.95,6.82,0,0.0,0.11,0.15,1231199999
2,7-Jan-09,0.0,0.0,50.0,0.0,0.0,0.0,0,1.0,0.0,...,1.12,1305.55,0.67,1763.60,6.84,0,0.0,0.11,0.47,1231372799
3,9-Jan-09,0.0,0.0,750.0,0.0,0.0,0.0,0,1.0,0.0,...,1.10,1324.45,0.66,1808.74,6.83,0,0.0,0.09,0.56,1231545599
4,11-Jan-09,0.0,0.0,7600.0,0.0,0.0,0.0,0,1.0,0.0,...,1.10,1324.45,0.66,1808.74,6.83,0,0.0,0.09,0.56,1231718399


In [17]:
research_data_imp2 = research_data.copy()
#research_data_imp2.sort_values(by='Date', inplace=True, ascending=True) # Sort by date ascending
forecast_out = int(math.ceil(0.0001 * len(research_data_imp2))) # 0.01% of data
#forecast_out = int(math.ceil(0.0025 * len(research_data_imp2))) # 0.05% of data
print(len(research_data_imp2))
print(forecast_out)

1773
1


In [18]:
research_data_imp2['Prediction'] = research_data_imp2['Close'].shift(-forecast_out)
research_data_imp2.tail()

,Date,Close,USD_Exchange_Trade_Volume,Bitcoins_in_circulation,MarketCap,BlockSize,AvgBlockSize,Orphaned_Block,TransactionsPerBlock,MedianTransactCnfmTime,...,USD/KRW,USD/GBP,EUR/KRW,USD/CNY,Google Trends Interest,GTrends Normalized,US Federal funds rate,Breakeven Inflation Rate,Epoch,Prediction
1768,9-Sep-18,6286.43,291261346.9,17259100.0,1.080000e+11,181951.87,0.62,0,1158.97,7.03,...,1128.55,0.77,1305.90,6.84,54,0.54,1.92,2.10,1536537599,6296.32
1769,11-Sep-18,6296.32,290980772.8,17262550.0,1.090000e+11,182206.04,0.94,0,1684.90,10.53,...,1125.80,0.77,1304.82,6.88,65,0.65,1.92,2.12,1536710399,6450.18
1770,13-Sep-18,6450.18,321882304.6,17266500.0,1.110000e+11,182446.38,0.82,0,1582.49,8.08,...,1120.89,0.76,1309.12,6.86,65,0.65,1.92,2.10,1536883199,6518.66
1771,15-Sep-18,6518.66,360631436.3,17269900.0,1.130000e+11,182696.55,0.79,0,1507.65,9.03,...,1121.43,0.77,1304.73,6.87,50,0.50,1.92,2.10,1537055999,6400.60
1772,17-Sep-18,6400.60,181083665.0,17273325.0,1.110000e+11,182906.28,0.91,0,1778.90,9.37,...,1124.70,0.76,1314.71,6.88,58,0.58,1.92,2.10,1537228799,NaN


In [19]:
def load_data():
    X = np.array(research_data_imp2.drop(['Prediction', 'Symbol', 'Date', 'Popularity'], 1)) # Create X with numeric columns only
    # X = np.array(research_data_imp2['Close'])
    # X = research_data_imp2['Close'].to_frame()
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:]
    X = X[:-forecast_out]

    # research_data_imp2.dropna(inplace=True)

    y = np.array(research_data_imp2['Prediction'])
    y = y[:-forecast_out]

    ##X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

    # split data into training set and a temporary set using sklearn.model_selection.traing_test_split
    #X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.2, random_state=23)
    
    # take the remaining 20% of data in X_tmp, y_tmp and split them evenly
    #X_test, X_val, y_test, y_val = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=23)
    
    #return [X_train, X_test, X_val, y_train, y_test, y_val]
    
    train_size = int(len(X) * .75)
    X_train, X_test = X[0:train_size], X[train_size:len(X)]
    y_train, y_test = y[0:train_size], y[train_size:len(y)]
    
    return [X_train, X_test, y_train, y_test]

In [20]:
def load_data_with_3D_array():
    #input_cols = ['Open', 'High', 'Low', 'Close', 'Volume BTC', 'Volume USD', 'Popularity_Normalized', 'Epoch']
    #input_cols = ['Close', 'Popularity_Normalized']
    input_cols = ['Close']
    
    df = research_data_imp2.copy()
    #df.dropna(inplace=True)
    
    # Put your inputs into a single list
    df['single_input_vector'] = df[input_cols].apply(tuple, axis=1).apply(list)
    
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
    
    # Use .cumsum() to include previous row vectors in the current row list of vectors
    df['cumulative_input_vectors'] = df.single_input_vector.cumsum()
    
    # If your output is multi-dimensional, you need to capture those dimensions in one object
    # If your output is a single dimension, this step may be unnecessary
    # df['output_vector'] = df[output_cols].apply(tuple, axis=1).apply(list)
    
    # Pad your sequences so they are the same length
    max_sequence_length = df.cumulative_input_vectors.apply(len).max()
    
    # Save it as a list   
    padded_sequences = pad_sequences(df.cumulative_input_vectors.tolist(), max_sequence_length, dtype='float32').tolist()
    df['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)
    
    # print(df)
    
    # Extract your training data
    X_train_init = np.asarray(df.padded_input_vectors)
    
    # Use hstack to and reshape to make the inputs a 3d vector
    X = np.hstack(X_train_init).reshape(len(df),max_sequence_length,len(input_cols))
    #y_train = np.hstack(np.asarray(df.output_vector)).reshape(len(df),len(output_cols))
    y = np.hstack(np.asarray(df.Prediction)).reshape(len(df),1)
    
    return [df, X, y]
    
    #train_size = int(len(X) * .75)
    #X_train, X_test = X[0:train_size], X[train_size:len(X)]
    #y_train, y_test = y[0:train_size], y[train_size:len(y)]
    
    #return [X_train, X_test, y_train, y_test]

In [21]:
# (X_train, X_test, X_val, y_train, y_test, y_val) = load_data()
# print(X_train.shape)
# print(X_test.shape)
# print(X_val.shape)
# print(y_train.shape)
# print(y_test.shape)
# print(y_val.shape)

# (X_train, X_test,y_train, y_test) = load_data()

#(X_train, X_test,y_train, y_test) = load_data_with_3D_array()

#research_data_imp2.drop(research_data_imp2.tail(1).index,inplace=True)
#research_data_imp2.iloc[1807]['Prediction']
#research_data_imp2.set_value(0, 'Prediction', 6793.5)
#research_data_imp2.set_value(1807, 'Prediction', 445.56)
(df, X_train, y_train) = load_data_with_3D_array()
df

,Date,Close,USD_Exchange_Trade_Volume,Bitcoins_in_circulation,MarketCap,BlockSize,AvgBlockSize,Orphaned_Block,TransactionsPerBlock,MedianTransactCnfmTime,...,USD/CNY,Google Trends Interest,GTrends Normalized,US Federal funds rate,Breakeven Inflation Rate,Epoch,Prediction,single_input_vector,cumulative_input_vectors,padded_input_vectors
0,3-Jan-09,0.00,0.0,50.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.83,0,0.00,0.08,0.150,1231027199,0.00,[[0.0]],[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
1,5-Jan-09,0.00,0.0,50.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.82,0,0.00,0.11,0.150,1231199999,0.00,[[0.0]],"[[0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2,7-Jan-09,0.00,0.0,50.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.84,0,0.00,0.11,0.470,1231372799,0.00,[[0.0]],"[[0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
3,9-Jan-09,0.00,0.0,750.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.83,0,0.00,0.09,0.560,1231545599,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
4,11-Jan-09,0.00,0.0,7600.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.83,0,0.00,0.09,0.560,1231718399,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
5,13-Jan-09,0.00,0.0,17800.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.84,0,0.00,0.10,0.590,1231891199,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
6,15-Jan-09,0.00,0.0,30450.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.84,0,0.00,0.18,0.450,1232063999,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
7,17-Jan-09,0.00,0.0,41650.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.84,0,0.00,0.19,0.450,1232236799,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
8,19-Jan-09,0.00,0.0,52650.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.84,0,0.00,0.19,0.565,1232409599,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
9,21-Jan-09,0.00,0.0,64100.0,0.000000e+00,0.00,0.00,0,1.00,0.00,...,6.84,0,0.00,0.23,0.580,1232582399,0.00,[[0.0]],"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."


In [28]:
def create_lstm():
    model = Sequential()
    
#     model.add(LSTM(32, input_shape=(1, 8), return_sequences=True))
#     #model.add(Dropout(0.4))
#     #model.add(LSTM(16, return_sequences=False))
#     model.add(Dense(1))
#     model.add(Activation('tanh'))
#     optimizer = RMSprop(lr=0.005, clipvalue=1.)
    
    #model.add(LSTM(512, input_shape=(1, 8), dropout_U = 0.2, dropout_W = 0.2, return_sequences=True))
    #model.add(LSTM(256, return_sequences=True))
    #model.add(Dropout(0.5))
    #model.add(LSTM(128, return_sequences=True))
    #model.add(LSTM(64))
    #model.add(Dropout(0.5))
    
    #model.add(LSTM(512, input_shape=(1808, 1), dropout_U = 0.2, dropout_W = 0.2, return_sequences=True))
    #model.add(LSTM(256, return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(LSTM(64, input_shape=(1773, 1), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32, return_sequences=True))
    model.add(LSTM(16, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = RMSprop(lr=0.0001, decay=3e-8)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

lstm = create_lstm()
lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1773, 64)          16896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 1773, 64)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 1773, 32)          12416     
_________________________________________________________________
lstm_9 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 32,465
Trainable params: 32,465
Non-trainable params: 0
_________________________________________________________________


In [29]:
#X_train2 = np.reshape(X_train, (X_train.shape[0], X_train.shape[0] , X_train.shape[1])) #X.reshape(samples, timesteps, features)
#X_test2 = np.reshape(X_test, (X_test.shape[0], X_test.shape[0], X_test.shape[1]))
#X_val2 = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

#print(X_train2.shape)
#print(X_test2.shape)
#print(X_val2.shape)

# X_train2 = np.transpose(X_train)
# X_train2 = np.reshape(X_train2, (X_train2.shape[0], X_train2.shape[1], 1)) #X.reshape(samples, timesteps, features)
# print(X_train2.shape)

# X_test2 = np.transpose(X_test)
# X_test2 = np.reshape(X_test2, (X_test2.shape[0], X_test2.shape[1], 1)) #X.reshape(samples, timesteps, features)
# print(X_test2.shape)
# print(y_train.shape)

# (C_train, C_test, d_train, d_test) = load_data()

# C_train2 = np.reshape(C_train, (1, C_train.shape[0], C_train.shape[1])) #X.reshape(samples, timesteps, features)
# d_train2 = np.reshape(d_train, (1, -1)) #X.reshape(samples, timesteps, features)

# print(C_train2.shape)
# print(d_train2.shape)
# print(C_train2)
# print(d_train2)
# print(len(d_train2))

In [ ]:
#lstm.fit(X_train2, y_train, batch_size=128, epochs=50, validation_data=[X_val2, y_val])

lstm.fit(X_train, y_train, validation_split=0.25, epochs=5, batch_size=32)

Train on 1329 samples, validate on 444 samples
Epoch 1/5


In [ ]:
print(X_train.shape)
print(y_train.shape)